In [1]:
import time
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime

_time = time.time()
def date_to_str(date):
    date = str(date).split("-")
    return date[1] + "/" + date[2] 

def day_val(day):
    val = {"Monday":0,"Tuesday":1,"Wednesday":2,'Thursday':3,"Friday":4,"Saturday":5,"Sunday":6}
    return val[day]
xls = pd.ExcelFile("D:\Quynh\PAYROLL\Input file\Costing examples with new condition.xlsx")
def get_excel():
    time_df = pd.read_excel(xls, 'Check') 
    return time_df

def get_holiday():

    holiday_df = xls.parse("Holiday")
    holiday_list = [("" if m>9 else "0") + str(m) + "/" + ("" if d>9 else "0") + str(d) for d, m in zip(holiday_df["Day"], holiday_df["Month"])]

    return holiday_list

def get_holiday_rate():
    holiday_rate = 2.5
    return holiday_rate

def get_rate_data():
    rate1 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
    rate2 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
    rate3 = np.array([[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.7,1.7,1.7,1.7,2.2],[2.2,2.2,2.2,2.2,2.2]])   
    rate_data = {"fulltime": rate1, "parttime": rate2, "casual": rate3}
    return rate_data
def get_base_rate():
    base_rate = 20
    return base_rate
holiday_list = get_holiday()
holiday_rate = get_holiday_rate()

rate_data = get_rate_data()
def cal_base_rate(_type, day, start_time, end_time, rate_data,start_date,end_date):
    shift = 2      
    ord_rate = 0
    ot1_rate = 0
    ot2_rate = 0
    if date_to_str(start_date) in holiday_list:
        ord_rate = holiday_rate
        ot1_rate = holiday_rate
        ot2_rate = holiday_rate
    else:        
        if _type == "fulltime":
            if start_time < 5 or (end_time > 1 and start_date != end_date):
                shift = 2
            elif end_time > 19 or (end_time < 1 and start_date != end_date):
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        elif _type == "parttime":
            
            if start_time < 5 or (end_time > 1 and start_date != end_date):
                shift = 2
            elif end_time > 19 or (end_time < 1 and start_date != end_date):
                shift = 1
                             
            elif start_time >= 5 and end_time < 20:
                shift = 0

        else:
            
            if start_time < 5 or (end_time > 1 and start_date != end_date):
                shift = 2
            elif end_time > 19 or (end_time < 1 and start_date != end_date):
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        ord_rate = rate_data[_type][day_val(day)][shift]
        ot1_rate = rate_data[_type][day_val(day)][3]
        ot2_rate = rate_data[_type][day_val(day)][4]
    return (ord_rate,ot1_rate,ot2_rate,shift)
base_rate = get_base_rate()
def get_maxot1(_type):
    if _type == "casual":
        maxot_1 = 10
    elif _type == "fulltime" or _type == "parttime":
        maxot_1 = 8
    return maxot_1

time_df = get_excel()
def get_data(time_df):
    import re,math
    data= []
    last_data = []
    for row in time_df.iterrows():    
        _id = row[1]["employee_name"] 
        objecttimeid = row[1]['objecttimeid']
        parentotid = row[1]['ParentOTID']
        if type(parentotid) == float:
            parentotid = objecttimeid
        
        start_period = row[1]['PeriodStartDate'].date()   
        start = row[1]["RoundedStart"]
        end   = row[1]["RoundedEnd"]   
        start_time = start.hour + start.minute/60
        end_time = end.hour + end.minute/60
    
        _type = row[1]["employment_type"]
        for k in _type.lower().split('\n'):
            _type=re.sub(r"[^a-zA-Z0-9]+",'',k).replace("temporary","")
            
        start_date = start.date() 
        end_date   = end.date()
        
        start_day  = start.strftime('%A')
        end_day    = end.strftime('%A')  
            
        if ((start_date != end_date) and ( ((end_time >1) and (start_day in ["Friday","Saturday","Sunday"]))  or ( date_to_str(start_date) in holiday_list or date_to_str(end_date) in holiday_list ) )):     
            delta_time1 = 24 - start_time 
            delta_time2 = end_time      
            start_date1 = start_date   
            end_date1= end_date
            start_date2 = end_date
            end_date2= end_date
            parentotid1 = parentotid + '-1'
            parentotid2 = parentotid + '-2'
            start1= start
            end2=end
            end1= end.replace(hour=0,minute=0)
            start2=end1
            start_day1=start1.strftime('%A')
         
            start_day2=start2.strftime('%A')
            
            data.append((parentotid1, start1, end1, _type))
            data.append((parentotid2, start2, end2,_type))  
            last_data.append((_id,start_period,parentotid1,objecttimeid, start1, end1,start_day1, _type,delta_time1))
            last_data.append((_id,start_period,parentotid2, objecttimeid,start2, end2,start_day2, _type,delta_time2))
            
        else: 
            delta_time = (end-start).total_seconds() / 3600.0
    
            data.append((parentotid,start,end,_type ))
            last_data.append((_id,start_period,parentotid, objecttimeid,start, end,start_day, _type,delta_time))
        
    file = pd.DataFrame(data,columns=["ParentOT ID",'Start Date','End Date','Type'])

   
    for parrent_id in file["ParentOT ID"].unique():
        
        last_end_date = file[file["ParentOT ID"] == parrent_id]['End Date'].iloc[-1]
        last_start_date = file[file["ParentOT ID"] == parrent_id]['Start Date'].iloc[0]
        file.loc[file["ParentOT ID"] == parrent_id,"End Date"] = last_end_date 
        file.loc[file["ParentOT ID"] == parrent_id,"Start Date"] = last_start_date
    file['Day'] = [i.strftime('%A') for i in file['Start Date']]
    file['Start Time'] = [i.hour + i.minute/60 for i in file['Start Date']]
    file['End Time'] = [i.hour + i.minute/60 for i in file['End Date']]    
    file["Rate"] = [cal_base_rate(file['Type'][i],file['Day'][i],file['Start Time'][i],file['End Time'][i], rate_data,file['Start Date'][i].date(),file['End Date'][i].date()) for i in range(len(file['Day']))]
    last_file = pd.DataFrame(last_data,columns=["ID",'Start Period','Parent ID','Object ID','Start Date','End Date','Day','Type','Hour'])
    last_file['Rate'] = file["Rate"]
    last_file['Start Date'] = file['Start Date']
    last_file['End Date'] = file['End Date']
    table = pd.DataFrame(last_file.groupby(['Parent ID','ID','Start Period','Start Date','End Date','Day','Type','Rate'])['Hour'].sum()).reset_index()
        
    return table
    
get_data(time_df)



,Parent ID,ID,Start Period,Start Date,End Date,Day,Type,Rate,Hour
0,0F0F3534-5923-4583-ACA0-02DE25FAB078,Alex Thompson,2021-04-05,2021-04-10 08:00:00,2021-04-10 08:05:00,Saturday,fulltime,"(1.5, 1.5, 2.0, 0)",0.083333
1,25A65538-272E-4AD7-8D46-F7D4E01948ED,Quynh,2021-04-05,2021-04-11 08:00:00,2021-04-11 08:05:00,Sunday,casual,"(2.2, 2.2, 2.2, 0)",0.083333
2,330BD6B3-BF89-4068-990A-CEB467CE15A7,Aaron Edwards,2021-04-05,2021-04-10 04:30:00,2021-04-10 15:00:00,Saturday,casual,"(1.7, 1.7, 2.2, 2)",10.000000
3,4EF6A38A-E5BA-40DF-8E00-8D2C77C3D894,Aaron Edwards,2021-04-05,2021-04-08 04:30:00,2021-04-08 15:00:00,Thursday,parttime,"(1.25, 1.5, 2.0, 2)",10.000000
4,65DB7EEA-35E5-4375-A4EA-0047BB679436,Alex Thompson,2021-04-05,2021-04-08 14:30:00,2021-04-08 18:00:00,Thursday,casual,"(1.2, 1.7, 2.2, 0)",3.500000
5,78F3F7ED-EA4D-435F-8375-DB15FA75C71A,Aaron Edwards,2021-04-05,2021-04-06 04:30:00,2021-04-06 15:00:00,Tuesday,casual,"(1.45, 1.7, 2.2, 2)",10.000000
6,80F2CE93-F2B6-442C-9272-8B9014108A84,Aaron Edwards,2021-04-05,2021-04-11 04:30:00,2021-04-11 15:00:00,Sunday,casual,"(2.2, 2.2, 2.2, 2)",10.000000
7,8198FFA5-9318-48CB-AD4D-AB7FF46F4B51,Quynh,2021-03-22,2021-03-25 08:00:00,2021-03-25 15:00:00,Thursday,casual,"(1.2, 1.7, 2.2, 0)",6.000000
8,94C7FA56-9C5C-42E5-AFAA-88598EBA6373,Aaron Edwards,2021-04-05,2021-04-09 04:30:00,2021-04-09 16:15:00,Friday,casual,"(1.45, 1.7, 2.2, 2)",11.250000
9,A6EDA5DE-AAEB-462F-9317-039FCA0C6940,Aaron Edwards,2021-04-19,2021-04-19 04:30:00,2021-04-19 07:00:00,Monday,casual,"(1.45, 1.7, 2.2, 2)",2.500000


In [2]:
def cal_pay(_hour,_type, _rate, _ot1_rate, _ot2_rate):
    maxot_1 = get_maxot1(_type)
    if _hour <3:
        _hour =3
    return base_rate*(
        min([maxot_1,_hour])*_rate + 
        max([0,min([2,_hour - maxot_1])])*_ot1_rate + 
        max([0,_hour - maxot_1 - 2 ])*_ot2_rate)

    
def cal_payment():
    file = get_data(time_df)
    period_pay = {}
    shift_pay = []
    for row in file.iterrows():
        _id          = row[1]["ID"]
        _period      = row[1]["Start Period"] 
        _key_period  = _id + " " + str(_period)
        _hour = row[1]["Hour"]   
        _rate = row[1]["Rate"][0]
        _ot1_rate = row[1]["Rate"][1]
        _ot2_rate = row[1]["Rate"][2]
        _type = row[1]["Type"]
        _parent = row[1]["Parent ID"]
       
        total_hour = 0
        payment = 0
        maxot_1 = get_maxot1(_type) 
        

        if _key_period in period_pay:        
            total_hour, payment = period_pay[_key_period]

            if total_hour + min(_hour,maxot_1) <= 38:
                payment = cal_pay(_hour,_type, _rate, _ot1_rate, _ot2_rate)
            elif total_hour >= 40 :
                payment = base_rate*(max(_hour,3)*_ot2_rate)
            elif total_hour >=38 and total_hour + _hour <40:
                payment = base_rate*(3*_ot1_rate)

            elif total_hour >=38 and total_hour + _hour >=40: 
                payment = base_rate*((40-total_hour)*_ot1_rate + (max(_hour,3)+total_hour-40)*_ot2_rate)
            

            elif total_hour + min(_hour,maxot_1)  >  38 and total_hour < 38:
                ot_hour = (total_hour + _hour) - 38 
        
                base_hour = 38 - total_hour 
                 
                
                base_pay =base_rate*(base_hour *_rate)
            
                ot_pay = base_rate*(min(2,ot_hour)*_ot1_rate + max(0,ot_hour-2)*_ot2_rate)
                
                payment = base_pay + ot_pay
                                   
            total_hour += min(_hour,maxot_1)
              
            period_pay[_key_period] = (total_hour,payment) 
        else: 
            payment = cal_pay(_hour,_type, _rate, _ot1_rate, _ot2_rate)
            
            period_pay[_key_period] = (min(_hour,maxot_1),payment)
        
        shift_pay.append(payment)
    
        
    file["Pay"] = shift_pay
    
    return file


output = cal_payment()

result = output.drop(["Rate"],1).sort_values(['ID','Start Period','Start Date'])
print(result)
# result = result.to_excel("report_2405.xlsx", index = True)
print("Time elapsed: ", time.time() -  _time)

                                 Parent ID                 ID Start Period  \
15    C5EBB1C7-F9EF-46F6-9EDF-9C6CE628BF19      Aaron Edwards   2021-03-22   
16    D8855E78-9046-48F1-8735-92FCEE2110CA      Aaron Edwards   2021-04-05   
5     78F3F7ED-EA4D-435F-8375-DB15FA75C71A      Aaron Edwards   2021-04-05   
3     4EF6A38A-E5BA-40DF-8E00-8D2C77C3D894      Aaron Edwards   2021-04-05   
8     94C7FA56-9C5C-42E5-AFAA-88598EBA6373      Aaron Edwards   2021-04-05   
2     330BD6B3-BF89-4068-990A-CEB467CE15A7      Aaron Edwards   2021-04-05   
6     80F2CE93-F2B6-442C-9272-8B9014108A84      Aaron Edwards   2021-04-05   
9     A6EDA5DE-AAEB-462F-9317-039FCA0C6940      Aaron Edwards   2021-04-19   
4     65DB7EEA-35E5-4375-A4EA-0047BB679436      Alex Thompson   2021-04-05   
11    BAA8B23C-FAD0-40E5-BD3D-6F61EBA04B45      Alex Thompson   2021-04-05   
0     0F0F3534-5923-4583-ACA0-02DE25FAB078      Alex Thompson   2021-04-05   
17    F1D8D012-7368-4143-98F0-C0AFF63FB34E      Alex Thompson   